In [2]:
!pip install biopython



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.6 MB/s eta 0:00:00


In [3]:
# Sequence Alignment Workflow: Global, Local, and Multiple Alignment
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import Align
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Define sequences
seq1 = "ACCGTGA"
seq2 = "ACGTCGA"
seq3 = "ACCGAGA"

# GLOBAL ALIGNMENT (Needleman–Wunsch)
print("=== Global Alignment ===")
global_alignments = pairwise2.align.globalxx(seq1, seq2)
for alignment in global_alignments:
    print(format_alignment(*alignment))

# LOCAL ALIGNMENT (Smith–Waterman)
print("=== Local Alignment ===")
local_alignments = pairwise2.align.localxx(seq1, seq2)
for alignment in local_alignments:
    print(format_alignment(*alignment))

# MULTIPLE SEQUENCE ALIGNMENT (using pairwise scoring simulation)
print("=== Multiple Sequence Alignment (Illustrative) ===")
aligner = Align.PairwiseAligner()
sequences = [
    SeqRecord(Seq("ACCGTGA"), id="Seq1"),
    SeqRecord(Seq("ACGTCGA"), id="Seq2"),
    SeqRecord(Seq("ACCGAGA"), id="Seq3")
]

for i in range(len(sequences)):
    for j in range(i + 1, len(sequences)):
        score = aligner.score(sequences[i].seq, sequences[j].seq)
        print(f"{sequences[i].id} vs {sequences[j].id} = {score}")


=== Global Alignment ===
ACCGT-GA
| ||| ||
A-CGTCGA
  Score=6

ACCGT-GA
|| || ||
AC-GTCGA
  Score=6

=== Local Alignment ===
ACCGT-GA
| ||| ||
A-CGTCGA
  Score=6

ACCGT-GA
|| || ||
AC-GTCGA
  Score=6

=== Multiple Sequence Alignment (Illustrative) ===
Seq1 vs Seq2 = 6.0
Seq1 vs Seq3 = 6.0
Seq2 vs Seq3 = 5.0


/usr/local/lib/python3.12/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [4]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# Define the two sequence IDs you want to BLAST
sequence_ids = ["PU773469.1", "PU773470.1"] # Replace with your desired sequence IDs

# Perform BLAST for each sequence
for seq_id in sequence_ids:
    print(f"Performing BLAST for sequence ID: {seq_id}")
    # Perform the BLAST search
    result_handle = NCBIWWW.qblast(
        "blastn",  # BLAST program (nucleotide blast)
        "nt",      # Database (nucleotide database)
        seq_id     # Query sequence ID
    )

    # Parse the BLAST results
    blast_records = NCBIXML.parse(result_handle)

    # Print some information about the hits
    for blast_record in blast_records:
        print(f"Query: {blast_record.query}")
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                print(f"  Alignment: {alignment.title}")
                print(f"  Length: {alignment.length}")
                print(f"  E-value: {hsp.expect}")
                print(f"  Query start: {hsp.query_start}, Query end: {hsp.query_end}")
                print(f"  Subject start: {hsp.sbjct_start}, Subject end: {hsp.sbjct_end}")
                print(f"  Score: {hsp.score}")
                print(f"  Identities: {hsp.identities}")
                print(f"  Gaps: {hsp.gaps}")
                print(f"  Query: {hsp.query[0:75]}...")
                print(f"  Match: {hsp.match[0:75]}...")
                print(f"  Sbjct: {hsp.sbjct[0:75]}...")
                print("-" * 80)

    result_handle.close()

Performing BLAST for sequence ID: PU773469.1
Query: Sequence 35 from patent US 12098131 B2
Performing BLAST for sequence ID: PU773470.1
Query: Sequence 36 from patent US 12098131 B2
  Alignment: gi|3020765241|ref|XM_075101328.1| PREDICTED: Phalacrocorax aristotelis MAF bZIP transcription factor (MAF), transcript variant X5, mRNA
  Length: 1887
  E-value: 4.64427
  Query start: 1, Query end: 21
  Subject start: 783, Subject end: 763
  Score: 42.0
  Identities: 21
  Gaps: 0
  Query: TCAGATCGAAGTCATTAACAT...
  Match: |||||||||||||||||||||...
  Sbjct: TCAGATCGAAGTCATTAACAT...
--------------------------------------------------------------------------------
  Alignment: gi|2853416636|ref|XR_011345929.1| PREDICTED: Narcine bancroftii transcription factor Maf-like (LOC138745038), transcript variant X9, misc_RNA
  Length: 2097
  E-value: 4.64427
  Query start: 1, Query end: 21
  Subject start: 672, Subject end: 652
  Score: 42.0
  Identities: 21
  Gaps: 0
  Query: TCAGATCGAAGTCATTAACAT...
  Mat

In [5]:
def read_fasta(file_path):
    """Reads a FASTA file and returns a dictionary of sequences."""
    sequences = {}
    current_sequence_name = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                current_sequence_name = line[1:]
                sequences[current_sequence_name] = ''
            elif current_sequence_name:
                sequences[current_sequence_name] += line
    return sequences

def simple_sequence_comparison(seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-2):
    """Performs a simple character-by-character comparison of two sequences."""
    score = 0
    alignment_seq1 = ""
    alignment_seq2 = ""
    i, j = 0, 0

    while i < len(seq1) and j < len(seq2):
        if seq1[i] == seq2[j]:
            score += match_score
            alignment_seq1 += seq1[i]
            alignment_seq2 += seq2[j]
            i += 1
            j += 1
        else:
            # Simple approach: penalize mismatches and advance both
            score += mismatch_penalty
            alignment_seq1 += seq1[i]
            alignment_seq2 += seq2[j]
            i += 1
            j += 1

    # Add remaining characters with gap penalty
    while i < len(seq1):
        score += gap_penalty
        alignment_seq1 += seq1[i]
        alignment_seq2 += '-'
        i += 1
    while j < len(seq2):
        score += gap_penalty
        alignment_seq1 += '-'
        alignment_seq2 += seq2[j]
        j += 1

    return score, alignment_seq1, alignment_seq2

# Example usage:
# Create dummy FASTA files for demonstration
with open('sequence1.fasta', 'w') as f:
    f.write('>SeqA\n')
    f.write('ATGCGTACGTAC')

with open('sequence2.fasta', 'w') as f:
    f.write('>SeqB\n')
    f.write('ATGCGTAGCTAC')

# Read sequences from FASTA files
sequences1 = read_fasta('sequence1.fasta')
sequences2 = read_fasta('sequence2.fasta')

# Assuming each file has only one sequence for this example
seq1_name = list(sequences1.keys())[0]
seq2_name = list(sequences2.keys())[0]
sequence1 = sequences1[seq1_name]
sequence2 = sequences2[seq2_name]

# Perform simple comparison
score, aligned_seq1, aligned_seq2 = simple_sequence_comparison(sequence1, sequence2)

print(f"Sequence 1 ({seq1_name}): {sequence1}")
print(f"Sequence 2 ({seq2_name}): {sequence2}")
print("\nSimple Comparison Results:")
print(f"Alignment Score: {score}")
print(f"Aligned Sequence 1: {aligned_seq1}")
print(f"Aligned Sequence 2: {aligned_seq2}")

Sequence 1 (SeqA): ATGCGTACGTAC
Sequence 2 (SeqB): ATGCGTAGCTAC

Simple Comparison Results:
Alignment Score: 8
Aligned Sequence 1: ATGCGTACGTAC
Aligned Sequence 2: ATGCGTAGCTAC
